In [10]:
import warnings
def fxn(): 
	warnings.warn("deprecated",DeprecationWarning)

with warnings.catch_warnings( ):
    warnings.simplefilter("ignore")
    fxn( )
    
import numpy as np
import h5py

In [11]:
A01T = h5py.File('A01T_slice.mat', 'r')
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
y = y[0,0:X.shape[0]:1]
y = np.asarray(y, dtype=np.int32)
X = np.nan_to_num(X)

In [12]:
print(X.shape)
print(y.shape)
y -= 769
print(y)

(288, 25, 1000)
(288,)
[3 2 1 0 0 1 2 3 1 2 0 0 0 3 1 1 0 0 2 0 1 3 3 2 0 3 3 1 3 3 1 0 1 2 2 2 3
 2 0 3 1 2 1 2 3 1 2 0 0 0 3 1 0 2 0 2 1 3 0 2 2 0 2 1 3 3 3 2 0 3 1 3 1 0
 2 1 0 2 2 0 2 3 3 1 0 1 3 1 3 2 1 1 1 2 3 0 1 3 0 2 2 3 0 0 2 1 3 3 3 1 0
 2 1 3 0 3 2 1 3 3 0 1 1 2 3 1 0 0 3 1 0 2 1 1 2 0 3 2 2 2 2 0 1 0 1 0 0 2
 2 1 2 3 0 3 0 0 1 3 2 1 3 2 3 2 3 1 1 3 0 1 1 1 2 3 0 3 0 2 0 3 0 2 0 1 2
 2 3 0 1 3 1 2 2 0 3 1 3 0 0 2 2 1 3 1 1 0 1 3 3 1 1 1 1 3 3 2 3 0 1 2 1 0
 3 0 3 0 0 0 0 2 2 3 1 2 2 2 3 2 0 2 0 3 1 3 3 2 3 3 2 1 3 2 0 1 1 1 2 1 3
 2 3 1 2 0 3 0 2 3 0 2 0 1 1 0 3 0 3 2 2 0 2 1 1 0 2 0 1 0]


In [13]:
from keras.models import Sequential
from keras.layers import Permute, Flatten, Softmax, Dense, Conv1D, Conv2D, Conv2DTranspose, AveragePooling2D, Activation, Reshape
from keras.backend import permute_dimensions

In [14]:
model = Sequential()
model.add(Conv2D(filters=40, kernel_size=(22,1), activation='elu', input_shape=(1000, 22, 1), strides=1, data_format="channels_last"))
print(model.output_shape)
model.add(Permute((3, 2, 1)))
print(model.output_shape)
model.add(Conv2D(filters=40, kernel_size=(22, 1), activation='elu', data_format="channels_first"))
print(model.output_shape)
model.add(Permute((3, 2, 1)))
print(model.output_shape)
model.add(AveragePooling2D(pool_size=(75,1), strides=(15,1)))
print(model.output_shape)
model.add(Flatten())
model.add(Dense(units=4, kernel_initializer='glorot_normal'))
print(model.output_shape)

(None, 979, 22, 40)
(None, 40, 22, 979)
(None, 40, 1, 979)
(None, 979, 1, 40)
(None, 61, 1, 40)
(None, 4)


In [ ]:
from keras.optimizers import Adam
Adam = Adam(lr=0.15)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
one_hot = [[] for val in y]
for idx, val in enumerate(y):
    if val == 769:
        cur = [1, 0, 0, 0]
    elif val == 770:
        cur = [0, 1, 0, 0]
    elif val == 771:
        cur = [0, 0, 1, 0]
    else:
        cur = [0, 0, 0, 1]
    one_hot[idx] = cur
one_hot = np.asarray(one_hot)
print(one_hot.shape)
print(X.shape)
print(X.transpose(0, 2,1)[:, 0:22, :, None].shape)
print(one_hot[0:22].shape)




model.fit(X.transpose(0,2,1)[:, :, 0:22, None], y, epochs=10, batch_size=128)

(288, 4)
(288, 25, 1000)
(288, 22, 25, 1)
(22, 4)
Epoch 1/10
288/288 [==============================] - 14s 47ms/step - loss: 1.3863 - acc: 0.2500
Epoch 2/10
288/288 [==============================] - 14s 50ms/step - loss: 1.3863 - acc: 0.2500
Epoch 3/10
288/288 [==============================] - 15s 51ms/step - loss: 1.3863 - acc: 0.2500
Epoch 4/10
288/288 [==============================] - 15s 53ms/step - loss: 1.3863 - acc: 0.2500
Epoch 5/10
